In [117]:
print("j")


j


In [118]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone 
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import langchain_pinecone

In [119]:
PINECONE_API_KEY = "pcsk_nswHo_DJXnx8nb4NCeKEdv8BhZHs8JAnDrt7MhDRFJGYufsA4GY8yrYvn6HiCJxtVJbhb"
PINECONE_API_ENV = "gcp-starter"

In [120]:
import os
os.chdir("..")

print("Current working directory: ",os.getcwd())

Current working directory:  C:\Users\Subhro\Desktop\newproject


In [121]:
import os
os.chdir(r"C:\Users\Subhro\Desktop\newproject\End-to-end-Medical-Chatbot-using-Llama2")
print("Currently:",os.getcwd())

Currently: C:\Users\Subhro\Desktop\newproject\End-to-end-Medical-Chatbot-using-Llama2


In [122]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [123]:
import os
os.chdir(r"C:\Users\Subhro\Desktop\newproject\End-to-end-Medical-Chatbot-using-Llama2")
print("Currently:",os.getcwd())

Currently: C:\Users\Subhro\Desktop\newproject\End-to-end-Medical-Chatbot-using-Llama2


In [124]:
extracted_data = load_pdf(data='data/')

In [125]:
 #extracted_data

In [126]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [127]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 16594


In [128]:
# text_chunks

In [129]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [130]:
embeddings = download_hugging_face_embeddings()

In [131]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [132]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [133]:
# query_result

In [135]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [136]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name ="medical"
pc=Pinecone(api_key="pcsk_6cLNH7_BgtCnvCvMiYvT5ZikGHVoqpMuJZjv1LMRhQBRx4Xws1Tni5XzBJ3Jrwfm5SLVv3")
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension = 384,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws',region="us-east-1")
    )

In [191]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


KeyboardInterrupt: 

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings,
)


In [ ]:
docsearch

In [ ]:
retriver = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

In [ ]:
import os
os.environ["MISTRAL_API_KEY"] = "W6sTfEeTeTQtwfxhfNjK9aK5M0OU1NHH"

In [ ]:
from langchain_mistralai import ChatMistralAI

# Initialize Mistral AI LLM
llm = ChatMistralAI(
    model="mistral-small",  # Choose appropriate model
    temperature=0.4,
    max_tokens=500
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate



system_prompt=(

    "You are an assistant for the question asnwering task"
    "Search the both given pdf to extract data precisely"
    "Give answers miexd from both of the pdf"
    "Make the answer short and knowlegeful"
    "if you dont know say i dont know the answer"
    "Don't add gibbrish answers"
    "Give answers in humanize form"
    "answer concise"
    "\n\n"
    "{context}"
)


prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}"),
    ]
)

In [ ]:
question_answer_chain=create_stuff_documents_chain(llm, prompt)
rag_chain=create_retrieval_chain(retriver , question_answer_chain)

In [193]:

from flask import Flask, render_template, jsonify, request
app = Flask(__name__)

In [196]:
# Ensure no other route has the same name
@app.route("/", methods=["GET"])
def home():
    return render_template('chat.html')

@app.route("/get", methods=["GET", "POST"])
def chat():
    try:
        msg = request.form.get("msg", "")
        print("User Input:", msg)
        response = rag_chain.invoke({"input": msg})
        print("Response:", response.get("answer", "No answer found."))
        return str(response.get("answer", "Sorry, no response."))
    except Exception as e:
        print("Error during chat:", str(e))
        return "An error occurred. Please try again."


AssertionError: View function mapping is overwriting an existing endpoint function: chat

In [ ]:
response = rag_chain.invoke({"input": "what is acne medication"})
print(response["answer"])

Based on the provided GALE Encyclopedia of Medicine entry, the medication for acne can include topical treatments such as retinoids, antibiotics, and benzoyl peroxide, as well as systemic medications like oral antibiotics and isotretinoin. The choice of medication depends on the severity and persistence of acne, as well as the patient's overall health and potential allergies or sensitivities. It's important to consult a healthcare provider for a proper diagnosis and treatment plan.
